<a href="https://colab.research.google.com/github/andrewgray11/msds432finalproject/blob/main/Chicago_lat_long_to_Community_Areas_API_calls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import requests
import json

def get_community_area(lat, lng):
    # Define the API endpoint and request parameters
    url = "https://data.cityofchicago.org/resource/igwz-8jzy.json"
    params = {
        "$where": f"within_circle(location, {lat}, {lng}, 100)"
    }
    headers = {
        "X-App-Token": "b4aeprbmm7gsyjf1jede4pe44"
    }

    # Send the POST request to the API endpoint
    response = requests.post(url, headers=headers, params=params)

    # Parse the JSON response and extract the community area number and name
    try:
        data = json.loads(response.text)
        community_area_number = data[0]["area_numbe"]
        community_area_name = data[0]["community"]
        return community_area_number, community_area_name
    except:
        return None

# Open the input CSV file and create the output CSV file
with open("/content/New_Construction_Permits_20-23.csv", "r") as input_file, open("output_file.csv", "w", newline="") as output_file:
    reader = csv.DictReader(input_file)
    fieldnames = reader.fieldnames + ["community_area_number", "community_area_name"]
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    writer.writeheader()

    # Find the latitude and longitude columns
    lat_column = "LATITUDE" if "LATITUDE" in reader.fieldnames else None
    lng_column = "LONGITUDE" if "LONGITUDE" in reader.fieldnames else None
    if not lat_column or not lng_column:
        raise ValueError("Latitude and/or longitude column not found")

    # Iterate through each row in the input CSV file
    for row in reader:
        lat = row[lat_column]
        lng = row[lng_column]

        # Check that the latitude and longitude values are not empty or missing
        if lat and lng:
            try:
                lat = float(lat)
                lng = float(lng)
            except ValueError:
                continue # Skip this row if latitude or longitude cannot be converted to a float

        # Retrieve the community area for the lat/long pair
        result = get_community_area(lat, lng)
        if result:
            row["community_area_number"] = result[0]
            row["community_area_name"] = result[1]
        else:
            row["community_area_number"] = ""
            row["community_area_name"] = ""

        # Write the updated row to the output CSV file
        writer.writerow(row)


In [ ]:
import requests
import json
import base64

def get_community_area(lat, lng, creds):
    # Define the API endpoint and request parameters
    url = "https://data.cityofchicago.org/resource/igwz-8jzy.json"
    params = {
        "$where": f"within_circle(location, {lat}, {lng}, 100)"
    }
    headers = {
        "Authorization": f"Basic {creds}",
        "X-App-Token": "ffxzcvmigrRn4GPnQbXtJagW6"
        
    }

    # Send the POST request to the API endpoint
    response = requests.post(url, headers=headers, params=params)
    print(response.json())
    # Parse the JSON response and extract the community area number and name
    try:
        data = json.loads(response.text)
        community_area_number = data[0]["area_num_1"]
        community_area_name = data[0]["community"]
        return community_area_number, community_area_name
    except:
        return None

# Example usage
lat_lng_pairs = [
    (41.883811, -87.631749),
    (41.881832, -87.625923),
    (41.881876, -87.627459)
]

for lat, lng in lat_lng_pairs:
    result = get_community_area(lat, lng, "YW5kcmV3Z3JheTIwMjJAdS5ub3J0aHdlc3Rlcm4uZWR1OkRDbjNKIzY0ZDJpdQ==")
    if result:
        print(f"Lat/long ({lat}, {lng}) is in community area {result[0]} ({result[1]})")
    else:
        print(f"Unable to determine community area for lat/long ({lat}, {lng})")


{'error': True, 'message': "User d22e-jwf4 doesn't have access to view igwz-8jzy"}
Unable to determine community area for lat/long (41.883811, -87.631749)
{'error': True, 'message': "User d22e-jwf4 doesn't have access to view igwz-8jzy"}
Unable to determine community area for lat/long (41.881832, -87.625923)
{'error': True, 'message': "User d22e-jwf4 doesn't have access to view igwz-8jzy"}
Unable to determine community area for lat/long (41.881876, -87.627459)


In [ ]:
pip install sodapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sodapy import Socrata
import json

def get_community_area(lat, lng):
    # Define the API endpoint and request parameters
    client = Socrata("data.cityofchicago.org", "ffxzcvmigrRn4GPnQbXtJagW6") 
    params = {
        "$where": f"within_circle(location, {lat}, {lng}, 100)"
    }

    # Send the GET request to the API endpoint
    try:
        data = client.get("igwz-8jzy", params=params)
        json_response = json.dumps(data, indent=2) # get the JSON response
        print(json_response) # print the JSON response
    except:
        print(json_response)
# Example usage
lat_lng_pairs = [
    (41.883811, -87.631749),
    (41.881832, -87.625923),
    (41.881876, -87.627459)
]

for lat, lng in lat_lng_pairs:
    get_community_area(lat, lng)


UnboundLocalError: ignored

In [ ]:
import requests
from requests.auth import AuthBase
from sodapy import Socrata

# Define your OAuth2 credentials
username = 'andrewgray2022@u.northwestern.edu'
password = 'DCn3J#64d2iu'

# Define an OAuth2 authentication class
class OAuth2Auth(AuthBase):
    def __init__(self, access_token):
        self.access_token = access_token

    def __call__(self, r):
        r.headers['Authorization'] = 'Bearer ' + self.access_token
        return r

# Obtain an access token using your OAuth2 credentials
client = Socrata('data.cityofchicago.org', 'ffxzcvmigrRn4GPnQbXtJagW6', username=username, password=password, timeout=60)
token = client.get_new_token()

# Make an authenticated request using the access token
url = 'https://data.cityofchicago.org/resource/igwz-8jzy.json'
params = {'$limit': 10}
auth = OAuth2Auth(token['access_token'])
response = requests.get(url, params=params, auth=auth)
print(response.json())

AttributeError: ignored

In [ ]:
import requests
from sodapy import Socrata

# OAuth2 authentication details
client_id = 'b4aeprbmm7gsyjf1jede4pe44'
client_secret = '5p202ooisasatpp8ohpia906oubzgoyjth39h6irn7f1wqkdv5'
username = 'andrewgray2022@u.northwestern.edu'
password = 'DCn3J#64d2iu'
token_url = 'https://data.cityofchicago.org/oauth/token'

# Get an access token
auth = requests.auth.HTTPBasicAuth(client_id, client_secret)
data = {'grant_type': 'password', 'username': username, 'password': password}
headers = {'User-Agent': 'msds 432'}
response = requests.post(token_url, auth=auth, data=data, headers=headers)
access_token = response.json()['']

# Connect to the Socrata API with the access token
domain = 'data.cityofchicago.org'
dataset_id = 'igwz-8jzy'
client = Socrata(domain, access_token=access_token)
results = client.get(dataset_id, limit=10)
print(results)

KeyError: ignored